In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from utils import *
from fastapi import FastAPI, File, UploadFile
from fastapi.responses import JSONResponse

In [ ]:
with open('annotations_avatar_name_info.yaml', 'r') as fd:
    avatar_to_anns = { name: Bunch(name=name, **anns) for name,anns in yaml.safe_load(fd).items() }

with open('annotations_aid_avatars.yaml', 'r') as fd:
    aid_to_anns = { aid: Bunch(avatar_id=aid, **merge_anns(anns, avatar_to_anns.get(anns.get('name', None), {})))
                   for aid, anns in yaml.safe_load(fd).items()
                   }

len(avatar_to_anns), next(iter(avatar_to_anns.items())), len(aid_to_anns), next(iter(aid_to_anns.items()))

In [ ]:
with open('py_dataset_ms.json', 'r') as f:
  ms = json.load(f)

In [ ]:
for path, data in ms.items():
  if data is not None:
    aid = '%s:%s' % (data['targetPlayer']['id'], data['targetPlayer']['displayName'])
    extras = json.dumps(data)
    try:
      aid_to_anns[aid]['extras'] = extras
    except KeyError:
      aid_to_anns[aid] = {
        'avatar_id': avatar_id, 
        'name': data['targetPlayer']['displayName'],
        'creator': None,
        'url': None,
        'category': None,
        'extras': extras
      }

In [ ]:
class AviDB(SQLiteDB):

  def __init__(self, path='avi.db'):
    col_types = [
      ('avatar_id', 'text'),
      ('name', 'text'),
      ('creator', 'text'),
      ('url', 'text'),
      ('category', 'text'),
      ('extras', 'json')
    ]
    name = 'avi'
    super().__init__(path=path, name=name, col_types=col_types)
    
  def _create_table(self, col_types):
    self.execute('CREATE VIRTUAL TABLE IF NOT EXISTS %s USING fts4(%s)' % (self.name, ', '.join(['%s %s %s' % (c, t, 'UNIQUE' if i == 0 else '') for i, (c, t) in enumerate(col_types)])))
    
  def _create_index(self):
    pass

In [ ]:
avi_db = AviDB()

In [ ]:
avi_db.insert(aid_to_anns.values())

In [ ]:
avi_db.size()

In [ ]:
for avi in avi_db.iter_rows():
  print(avi)

In [ ]:
avi_db.get(2)

In [ ]:
list(avi_db.search('Rusk', 'name'))